In [59]:
import os
import openai
import streamlit as st

from pprint import pprint
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv() #loads all env vars

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
openai_client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [56]:

def save_documents(release_data):
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
    docs = []
    for data in release_data:   
        docs.extend(text_splitter.create_documents([data]))
    
    db = Chroma.from_documents(docs, embeddings)
    return db

# load documents 
def load_documents(folder_name):
    release_data = []
    
    files = os.listdir(folder_name)
    for file_name in files:
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_name, file_name)
            text_data = open(file_path, encoding="utf-8").read()
            release_data.append(text_data)              
    return release_data

def get_llm_response(context, query):
    prompt = f"Given the context: {context}, please answer the following question {query}."
    
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {'role': 'system', 'content': "You have to answer question based on context given"},
            {'role': 'user', 'content': prompt}
        ],
        temperature=0
    )
    return response.choices[0].message.content

def main():
    release_data_list = load_documents('data')
    chroma_db = save_documents(release_data_list)

    # title of app
    st.title("T-systems Q&A")

    # sidebar for settings
    st.sidebar.title("Settings")
    api_key = st.sidebar.text_input("Enter you openAI Api key: ", type="password")

    # response param
    temperature = st.sidebar.slider("Temperature", min_value=0.0, max_value=1.0, value=0.5)

    # main interface for user input
    st.write("Go ahead and ask question related to Tsystem press release")
    query = st.text_input("Search")
    
    query = "Tell me something about Deutsche Telekom and Google Cloud partnership"
    retriver = chroma_db.as_retriever()
    # print(retriver.get_relevant_documents(query)[0].page_content)
    realase_data_relevant = retriver.get_relevant_documents(query)[0].page_content
    context = " ".join(realase_data_relevant)
    llm_response = get_llm_response(context, query)
    pprint(llm_response)


In [57]:
main()

('Deutsche Telekom and Google Cloud have announced an expanded partnership to '
 'collaborate on defining a joint roadmap for the telecommunications industry. '
 'This partnership aims to bring the power of the cloud closer to mobile and '
 "connected devices at the edge of Deutsche Telekom's network. By working "
 'together, they are reimagining network infrastructures to develop new '
 'network deployment models. This collaboration will focus on creating more '
 'secure, reliable, and scalable cloud-native networks, enabling companies '
 'like Deutsche Telekom to deliver elastic, high-capacity, high-bandwidth, and '
 'low-latency connectivity, as well as value-added services to their '
 'customers. The initial step of this partnership will involve piloting '
 'various network services, such as 5G Standalone in Austria and remote packet '
 "gateway functions, leveraging Google Cloud's technology and expertise.")
